In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [1]:
PATH='data/jigsaw-toxic-comment-classification/'
TRAIN=f'{PATH}train/'
TEST=f'{PATH}test/'
VALID=f'{PATH}valid/'

In [2]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle



    Only loading the 'en' tokenizer.



In [3]:
!mkdir -p {PATH}train
!mkdir -p {PATH}test
!mkdir -p {PATH}valid

In [4]:
!cp {PATH}/test.csv {TEST}
!cp {PATH}/train.csv {TRAIN}
!cp {PATH}/test.csv {VALID}valid.csv

In [5]:
train = pd.read_csv(f'{TRAIN}train.csv')
test = pd.read_csv(f'{TEST}test.csv')
valid = pd.read_csv(f'{VALID}valid.csv')

In [6]:
bs=16; bptt=70

In [7]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [8]:
data = LanguageModelData.from_dataframes(PATH, TEXT, 'comment_text', train, test, test_df=test, bs=bs, bptt=bptt)

In [9]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [10]:
len(data.trn_dl), data.nt, len(data.trn_ds), len(data.trn_ds[0].text)

(7157, 160150, 1, 8017384)

In [11]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '.', 'the', ',', '"', 'to', 'i', 'of', 'and', 'you', 'a']

In [12]:
TEXT.vocab.stoi['the']

3

In [13]:
data.trn_ds[0].text[:12]

['nonsense',
 '?',
 ' ',
 'kiss',
 'off',
 ',',
 'geek',
 '.',
 'what',
 'i',
 'said',
 'is']

In [14]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [15]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [16]:
learner = data.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [17]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

  0%|          | 1/7157 [00:01<3:52:19,  1.95s/it, loss=12]


RuntimeError: cuda runtime error (2) : out of memory at /home/bmn/src/ai.learn/pytorch/pytorch/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
learner.save_encoder('adam1_enc')

In [ ]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=10, cycle_save_name='adam3_10')

In [ ]:
learner.save_encoder('adam3_10_enc')

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

In [ ]:
learner.save_encoder('adam3_20_enc')

In [ ]:
m=learner.model
ss=""". Nonsense?  kiss off, geek. what I said is true.  I'll have"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

In [ ]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs